# Irony Detection Benchmark on TweetEval

## Objective

This notebook benchmarks several approaches for irony detection on Twitter
using the TweetEval Irony dataset.

We evaluate models based on a trade-off between **performance** (F1-Score on the Irony class) and **inference efficiency** (VRAM usage, Latency).

**Approaches Compared:**
1.  **State-of-the-Art (Reference):** `cardiffnlp/twitter-roberta-base-2021-124m-irony` 
2.  **Efficient Fine-Tuning (Challenger):** `SetFit` with `sentence-transformers/paraphrase-mpnet-base-v2`
3.  **Classic ML + Embeddings:** `XGBoost` on `Jina-Embeddings-v3` (dim reduced)
4.  **LLM (Baseline):** `Qwen 2.5 7B` (Local inference) 

## Dataset: TweetEval (Irony)

We use the official **TweetEval** benchmark (Irony subset), derived from *SemEval-2018 Task 3*.  
It consists of tweets labelled as **0 (Non-Irony)** or **1 (Irony)**.

**Data Splits & Distribution:**

| Split | Total Samples | Non-Irony (0) | Irony (1) | Balance (Irony %) |
| :--- | :---: | :---: | :---: | :---: |
| **Train** | 2,862 | ~1,430 | ~1,432 | ~50% (Balanced) |
| **Validation** | 955 | ~470 | ~485 | ~50% (Balanced) |
| **Test** | **784** | **473** | **311** | **39.7% (Imbalanced)** |

> **⚠️ Critical Observation:** > The training set is artificially balanced, but the **Test set reflects a more realistic distribution** (Irony is the minority class ~40%).  
> Therefore, `Accuracy` is misleading. We prioritize **F1-Score on the Positive Class (Irony)**.

**Metrics:**
* **Primary:** `F1-Score (Irony)` - Measures the ability to correctly identify irony without false positives.
* **Secondary:** `Accuracy` - Global performance indicator.
* **Engineering:** `Inference Speed` & `VRAM footprint`.

---

In [ ]:
import os
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb  

from datasets import load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers.utils import logging
from sklearn.metrics import accuracy_score, f1_score, classification_report

# --- 1. CONFIGURATION HARDWARE (RTX 4060 OPTIMIZATION) ---
# On fait taire les warnings non critiques de Transformers
logging.set_verbosity_error()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# La RTX 4060 supporte bfloat16 : c'est le standard actuel pour l'inférence efficace
DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

# Seed pour la reproductibilité (Indispensable pour un benchmark)
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Hardware: {torch.cuda.get_device_name(0) if DEVICE == 'cuda' else 'CPU'}")
print(f"Precision: {DTYPE}")
print(f"Device: {DEVICE}")

# --- 2. DATA INGESTION ---
# Chargement du benchmark officiel "tweet_eval" subset "irony"
print("\n📥 Chargement du dataset TweetEval (Irony)...")
dataset = load_dataset("tweet_eval", "irony")

# Extraction des splits
train_ds = dataset["train"]
val_ds = dataset["validation"]
test_ds = dataset["test"]

print(f"Dataset chargé avec succès.")
print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")
print(f"Labels: {train_ds.features['label'].names} (0=Not_Irony, 1=Irony)")

c:\Users\anass\Desktop\irony-detection-nlp-benchmark\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hardware: NVIDIA GeForce RTX 4060 Laptop GPU
Precision: torch.bfloat16
Device: cuda

📥 Chargement du dataset TweetEval (Irony)...
Dataset chargé avec succès.
Train: 2862 | Val: 955 | Test: 784
Labels: ['non_irony', 'irony'] (0=Not_Irony, 1=Irony)


In [3]:
# --- 3. HELPER FUNCTIONS & UTILS ---
import time
import gc
import re

# Global leaderboard to accumulate results for the final report
leaderboard = []

def flush_memory():
    """
    Aggressively cleans up VRAM between models.
    Crucial for RTX 4060 (8GB) to avoid OOM (Out Of Memory) errors.
    """
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.ipc_collect()
    print("Memory flushed. VRAM ready for the next round.")

def evaluate_model(y_true, y_pred, model_name, inference_time=None):
    """
    Standardized evaluation function.
    Ensures fair comparison across all models using the same metrics.
    """
    acc = accuracy_score(y_true, y_pred)
    f1_irony = f1_score(y_true, y_pred, pos_label=1)
    
    print(f"\n📊 RESULTS: {model_name}")
    print(f"  Accuracy: {acc:.2%}")
    print(f"  F1-Score (Irony): {f1_irony:.2%}")
    if inference_time:
        print(f"  Inference Time: {inference_time:.2f}s")
        
    leaderboard.append({
        "Model": model_name,
        "Accuracy": acc,
        "F1_Irony": f1_irony,
        "Inference_Time_s": inference_time
    })
    
    print("-" * 40)
    print(classification_report(y_true, y_pred, target_names=["Not_Irony", "Irony"]))
    print("=" * 40)


def preprocess(text):
    """
    Cardiff models expect @user and http normalization.
    """
    new_text = [
    ]
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# --- APPLY TO ALL SPLITS ---
def preprocess_dataset(dataset):
    """Apply preprocessing to a HuggingFace dataset split."""
    return dataset.map(lambda x: {"text": preprocess(x["text"])})

print("🔄 Applying preprocessing...")

train_ds = preprocess_dataset(train_ds)
val_ds = preprocess_dataset(val_ds)
test_ds = preprocess_dataset(test_ds)

print("Helper functions loaded. Ready to benchmark.")

🔄 Applying preprocessing...


Map: 100%|██████████| 784/784 [00:00<00:00, 13246.27 examples/s]

Helper functions loaded. Ready to benchmark.


# Cardiff twitter-roberta-base-2021-124m-irony

In [4]:
# =============================================================================
# EXPERIMENT 1: SOTA Reference — Cardiff RoBERTa (2021-124M)
# =============================================================================
# This is the state-of-the-art industrializable model for irony detection.
# Pre-trained on 124M tweets, then fine-tuned on TweetEval Irony.
# =============================================================================

MODEL_NAME_CARDIFF = "cardiffnlp/twitter-roberta-base-2021-124m-irony"

print("=" * 60)
print("EXPERIMENT 1: Cardiff RoBERTa (SOTA Reference)")
print("=" * 60)
print(f"Model: {MODEL_NAME_CARDIFF}")
print(f"Device: {DEVICE}")

# --- Load Model & Tokenizer ---
tokenizer_cardiff = AutoTokenizer.from_pretrained(MODEL_NAME_CARDIFF)
model_cardiff = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME_CARDIFF)
model_cardiff = model_cardiff.to(DEVICE).eval()

# --- Identify Irony class ID (robust method) ---
irony_id = 1  # default
for k, v in model_cardiff.config.id2label.items():
    label = str(v).lower()
    if "irony" in label and "non" not in label and "not" not in label:
        irony_id = int(k)
        print(f"✅ Irony class detected: id={irony_id} → '{v}'")

# --- Prepare Test Data ---
test_texts = test_ds["text"]
test_labels = np.array(test_ds["label"], dtype=np.int64)

print(f"Test samples: {len(test_texts)}")

# --- Inference with timing ---
BATCH_SIZE = 64
probas = []

print("\nRunning inference...")
start_time = time.time()

with torch.inference_mode():
    for i in range(0, len(test_texts), BATCH_SIZE):
        batch = test_texts[i:i + BATCH_SIZE]
        
        # Tokenize
        encoded = tokenizer_cardiff(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        ).to(DEVICE)
        
        # Forward pass
        logits = model_cardiff(**encoded).logits
        probs = torch.softmax(logits, dim=-1)[:, irony_id]
        probas.append(probs.cpu().numpy())

inference_time = time.time() - start_time

# --- Predictions ---
probas = np.concatenate(probas)
y_pred_cardiff = (probas >= 0.5).astype(np.int64)

print(f"✅ Inference complete in {inference_time:.2f}s")
print(f"   Throughput: {len(test_texts) / inference_time:.1f} samples/sec")

# --- Evaluation (uses helper function) ---
evaluate_model(test_labels, y_pred_cardiff, "Cardiff RoBERTa-2021-124M (SOTA)")

# --- Cleanup ---
del model_cardiff, tokenizer_cardiff
flush_memory()

EXPERIMENT 1: Cardiff RoBERTa (SOTA Reference)
Model: cardiffnlp/twitter-roberta-base-2021-124m-irony
Device: cuda
✅ Irony class detected: id=1 → 'irony'
Test samples: 784

Running inference...
✅ Inference complete in 1.72s
   Throughput: 457.1 samples/sec

📊 RESULTS: Cardiff RoBERTa-2021-124M (SOTA)
  Accuracy: 78.57%
  F1-Score (Irony): 75.07%
----------------------------------------
              precision    recall  f1-score   support

   Not_Irony       0.86      0.77      0.81       473
       Irony       0.70      0.81      0.75       311

    accuracy                           0.79       784
   macro avg       0.78      0.79      0.78       784
weighted avg       0.80      0.79      0.79       784

Memory flushed. VRAM ready for the next round.


# Cardiff twitter-roberta-base-irony

In [ ]:
# =============================================================================
# EXPERIMENT 1b: Cardiff RoBERTa (Base Version) — Comparison
# =============================================================================
# Older model, fine-tuned on less data than the 2021-124M version.
# Useful to see the impact of pre-training corpus size.
# =============================================================================

MODEL_NAME_CARDIFF_BASE = "cardiffnlp/twitter-roberta-base-irony"

print("=" * 60)
print("EXPERIMENT 1b: Cardiff RoBERTa (Base Version)")
print("=" * 60)
print(f"Model: {MODEL_NAME_CARDIFF_BASE}")
print(f"Device: {DEVICE}")

# --- Load Pipeline (simpler approach for this model) ---
pipe_cardiff_base = pipeline(
    "text-classification",
    model=MODEL_NAME_CARDIFF_BASE,
    tokenizer=MODEL_NAME_CARDIFF_BASE,
    device=0 if DEVICE == "cuda" else -1
)

# --- Prepare Test Data (already preprocessed) ---
test_texts = test_ds["text"]
test_labels = np.array(test_ds["label"], dtype=np.int64)

print(f"Test samples: {len(test_texts)}")

# --- Inference with timing ---
BATCH_SIZE = 32
predictions = []

print("\n🚀 Running inference...")
start_time = time.time()

for i in range(0, len(test_texts), BATCH_SIZE):
    batch = test_texts[i:i + BATCH_SIZE]
    results = pipe_cardiff_base(batch)
    
    for res in results:
        # Map text labels to integers
        # 'irony' -> 1, 'non_irony' -> 0
        label_str = res['label'].lower()
        label_int = 1 if "irony" in label_str and "non" not in label_str else 0
        predictions.append(label_int)

inference_time = time.time() - start_time

# --- Convert to array ---
y_pred_cardiff_base = np.array(predictions, dtype=np.int64)

print(f"✅ Inference complete in {inference_time:.2f}s")
print(f"   Throughput: {len(test_texts) / inference_time:.1f} samples/sec")

# --- Evaluation ---
evaluate_model(test_labels, y_pred_cardiff_base, "Cardiff RoBERTa-Base (Older)")

# --- Cleanup ---
del pipe_cardiff_base
flush_memory()

📊 EXPERIMENT 1b: Cardiff RoBERTa (Base Version)
Model: cardiffnlp/twitter-roberta-base-irony
Device: cuda
Test samples: 784

🚀 Running inference...
✅ Inference complete in 9.54s
   Throughput: 82.2 samples/sec

📊 RESULTS: Cardiff RoBERTa-Base (Older)
  Accuracy: 73.47%
  F1-Score (Irony): 62.72%
----------------------------------------
              precision    recall  f1-score   support

   Not_Irony       0.75      0.85      0.79       473
       Irony       0.71      0.56      0.63       311

    accuracy                           0.73       784
   macro avg       0.73      0.71      0.71       784
weighted avg       0.73      0.73      0.73       784

Memory flushed. VRAM ready for the next round.


### Comparative Analysis: Scale & Time Matter

**Correction on Model Specs:**
We are comparing two very different beasts. The performance gap is driven by two massive factors:

1.  **Data Volume (The biggest factor):**
    * **Legacy Model:** ~58 Million tweets.
    * **Modern Model:** ~124 Million tweets (**+113% more data**).
    * *Impact:* The model has seen twice as many examples of sarcasm structure, independent of time.

2.  **Temporal Freshness (Concept Drift):**
    * **Legacy:** Training stops ~2019.
    * **Modern:** Training covers up to Dec 2021.
    * *Impact:* Crucial for catching irony based on recent events (Covid, Crypto, Elections).

**The Verdict:**
* **Legacy Model:** F1-Score (Irony) = **62.7%**
* **Modern Model:** F1-Score (Irony) = **75.1%**

**Analysis:**
The **+12 point gain** is a combination of "More Data" (better generalization) and "Newer Data" (better context). The Legacy model is simply under-trained by modern standards.


# Fine-Tuning SetFit (MPNet)

In [8]:
# =============================================================================
# EXPERIMENT 2: SetFit (MPNet) — Challenger
# =============================================================================

MODEL_NAME_SETFIT = "sentence-transformers/paraphrase-mpnet-base-v2"
LABELS_NAMES = ["NOT_IRONY", "IRONY"]

print("=" * 60)
print("EXPERIMENT 2: SetFit (MPNet) — Challenger")
print("=" * 60)

# --- IMPORTANT: Reload RAW data (no preprocessing for MPNet) ---
print("Loading RAW dataset (no preprocessing for SetFit)...")
raw_dataset = load_dataset("tweet_eval", "irony")
train_raw = raw_dataset["train"]
val_raw = raw_dataset["validation"]
test_raw = raw_dataset["test"]

print(f"Model: {MODEL_NAME_SETFIT}")
print(f"Device: {DEVICE}")
print(f"Train: {len(train_raw)} | Val: {len(val_raw)} | Test: {len(test_raw)}")

# --- Model ---
model_setfit = SetFitModel.from_pretrained(
    MODEL_NAME_SETFIT,
    device=DEVICE,
    labels=LABELS_NAMES,
)

# --- Training Arguments ---
args = TrainingArguments(
    output_dir="setfit_checkpoints_irony",
    batch_size=16,
    num_epochs=1,
    num_iterations=25,
    body_learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="embedding_loss",
    greater_is_better=False,
    use_amp=(DEVICE == "cuda"),
    report_to="none",
    seed=SEED,
)

# --- Trainer (with RAW data) ---
trainer = Trainer(
    model=model_setfit,
    args=args,
    train_dataset=train_raw,  # RAW
    eval_dataset=val_raw,     # RAW
    metric="accuracy",
)

# --- Train ---
print("\n Training...")
start_train = time.time()
trainer.train()
train_time = time.time() - start_train
print(f"✅ Training complete in {train_time:.1f}s")


# --- Inference ---
print("\n🚀 Inference on test set...")
test_labels = np.array(test_ds["label"], dtype=np.int64)

start_inference = time.time()
raw_preds = model_setfit.predict(test_ds["text"])

# 🔧 FIX AUTOMATIQUE : Conversion String -> Int si nécessaire
# Si le modèle renvoie ['NOT_IRONY', 'IRONY'], on mappe vers [0, 1]
if len(raw_preds) > 0 and isinstance(raw_preds[0], str):
    label_map = {"NOT_IRONY": 0, "IRONY": 1, "not_irony": 0, "irony": 1}
    y_pred_setfit = np.array([label_map.get(str(p), 0) for p in raw_preds], dtype=np.int64)
else:
    # Si c'est déjà des chiffres, on passe direct
    y_pred_setfit = np.array(raw_preds, dtype=np.int64)

inference_time = time.time() - start_inference
print(f"✅ Inference complete in {inference_time:.2f}s")

# --- Evaluation ---
evaluate_model(test_labels, y_pred_setfit, "SetFit (MPNet)", inference_time)

# --- Save ---
SETFIT_SAVE_PATH = "models/setfit-mpnet-irony"
# On s'assure que le dossier existe pour éviter une erreur
os.makedirs(SETFIT_SAVE_PATH, exist_ok=True) 

EXPERIMENT 2: SetFit (MPNet) — Challenger
Loading RAW dataset (no preprocessing for SetFit)...
Model: sentence-transformers/paraphrase-mpnet-base-v2
Device: cuda
Train: 2862 | Val: 955 | Test: 784


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.



 Training...


***** Running training *****
  Num unique pairs = 143100
  Batch size = 16
  Num epochs = 1


{'embedding_loss': 0.4499, 'grad_norm': 2.9424524307250977, 'learning_rate': 2.2346368715083798e-08, 'epoch': 0.00011180679785330948}
{'embedding_loss': 0.3386, 'grad_norm': 2.0950355529785156, 'learning_rate': 1.11731843575419e-06, 'epoch': 0.005590339892665474}
{'embedding_loss': 0.2736, 'grad_norm': 1.3392945528030396, 'learning_rate': 2.23463687150838e-06, 'epoch': 0.011180679785330949}
{'embedding_loss': 0.258, 'grad_norm': 0.949404776096344, 'learning_rate': 3.35195530726257e-06, 'epoch': 0.01677101967799642}
{'embedding_loss': 0.2547, 'grad_norm': 1.3016060590744019, 'learning_rate': 4.46927374301676e-06, 'epoch': 0.022361359570661897}
{'embedding_loss': 0.2571, 'grad_norm': 1.3423534631729126, 'learning_rate': 5.58659217877095e-06, 'epoch': 0.02795169946332737}
{'embedding_loss': 0.2533, 'grad_norm': 0.9251423478126526, 'learning_rate': 6.70391061452514e-06, 'epoch': 0.03354203935599284}
{'embedding_loss': 0.2553, 'grad_norm': 0.9265353083610535, 'learning_rate': 7.821229050279

In [ ]:
path = "models/setfit-mpnet-irony"
model = SetFitModel.from_pretrained(path)

def count_torch_params(module) -> int:
    """Count PyTorch parameters."""
    if module is None or not hasattr(module, "parameters"):
        return 0
    return sum(p.numel() for p in module.parameters())

def count_sklearn_params(head) -> int:
    """Count sklearn LogisticRegression parameters."""
    if head is None:
        return 0
    n = 0
    if hasattr(head, "coef_"):
        n += head.coef_.size
    if hasattr(head, "intercept_"):
        n += head.intercept_.size
    return n

# --- Get components ---
body = getattr(model, "model_body", None)
head = getattr(model, "model_head", None)

# --- Count ---
n_body = count_torch_params(body)
n_head_torch = count_torch_params(head)
n_head_sklearn = count_sklearn_params(head)

# --- Display ---
print("=" * 50)
print("📊 SetFit Model Parameters")
print("=" * 50)
print(f"Body type: {type(body).__name__}")
print(f"Head type: {type(head).__name__}")
print("-" * 50)
print(f"Body (MPNet):     {n_body:>15,} params")
print(f"Head (sklearn):   {n_head_sklearn:>15,} params")
print("-" * 50)
print(f"TOTAL:            {n_body + n_head_sklearn:>15,} params")
print("=" * 50)

# Cleanup
del model

📊 SetFit Model Parameters
Body type: SentenceTransformer
Head type: LogisticRegression
--------------------------------------------------
Body (MPNet):         109,486,464 params
Head (sklearn):               769 params
--------------------------------------------------
TOTAL:                109,487,233 params


## Analysis: SetFit (MPNet) vs SOTA

### Performance Comparison

| Metric | SetFit (MPNet) | SOTA (RoBERTa-124M) | Gap |
|--------|----------------|---------------------|-----|
| **F1 (Irony)** | 72.00% | 75.07% | -3.07 pts |
| **Accuracy** | 74.00% | 78.57% | -4.57 pts |
| **Recall (Irony)** | 83.00% | 81.35% | **+1.65 pts**|
| **Parameters** | 109.5M | 125M | -12% |
| **Inference Time** | 0.96s | 1.72s | **1.8x faster**|

---

### Key Takeaways

| Aspect | SetFit | SOTA | Verdict |
|--------|--------|------|---------|
| **Training Time** | ~34 min | Pre-trained on 124M tweets | SetFit = from scratch |
| **Hyperparameter Tuning** | None (default values) | Optimized | Room for improvement |
| **Fine-tuning required** | Simple contrastive learning | Full fine-tuning | SetFit simpler |
| **GPU Memory** | ~4 GB | ~4 GB | Equivalent |

---

### SetFit Advantages

1. **Fast training** — 34 min on RTX 4060 (vs hours for classic fine-tuning)
2. **Few-shot capable** — Can work with very few examples
3. **No hyperparameter tuning** — Competitive results with default config
4. **Fast inference** — 1.8x faster than SOTA
5. **High recall** — Better at detecting irony cases (+1.65 pts)


# Classic ML
## Jina Embeddings v3 (512 dimensions)

In [11]:
# =============================================================================
# 🧹 AGGRESSIVE MEMORY CLEANUP
# =============================================================================
print("🔴 VRAM avant cleanup:")
if torch.cuda.is_available():
    print(f"   Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"   Reserved:  {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

# 1. Delete any remaining model variables
vars_to_delete = [
    'model_cardiff', 'tokenizer_cardiff', 'pipe_cardiff_base',
    'model_setfit', 'trainer', 'encoder',
    'best_rf', 'best_xgb', 'rf_search', 'xgb_search',
    'X_train', 'X_val', 'X_test', 'X_full_train',
    'raw_dataset', 'train_raw', 'val_raw', 'test_raw'
]

for var in vars_to_delete:
    if var in globals():
        del globals()[var]
        print(f"   Deleted: {var}")

# 2. Aggressive garbage collection
gc.collect()
gc.collect()
gc.collect()

# 3. Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    torch.cuda.synchronize()

# 4. Final check
print("\n✅ VRAM après cleanup:")
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"   Allocated: {allocated:.2f} GB")
    print(f"   Reserved:  {reserved:.2f} GB")
    print(f"   Free:      {total - reserved:.2f} GB")
    print(f"   Total:     {total:.2f} GB")

🔴 VRAM avant cleanup:
   Allocated: 1.65 GB
   Reserved:  7.40 GB
   Deleted: model_setfit
   Deleted: trainer
   Deleted: raw_dataset
   Deleted: train_raw
   Deleted: val_raw
   Deleted: test_raw

✅ VRAM après cleanup:
   Allocated: 0.43 GB
   Reserved:  0.80 GB
   Free:      7.19 GB
   Total:     8.00 GB


In [12]:
# =============================================================================
# EXPERIMENT 3: Classic ML + Jina Embeddings v3
# =============================================================================
# Approach: Extract embeddings with Jina v3, then train XGBoost/RF classifiers.
# This tests if high-quality embeddings + classic ML can compete with transformers.
# =============================================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer
import joblib

print("=" * 60)
print("EXPERIMENT 3: Jina v3 Embeddings + Classic ML")
print("=" * 60)

# --- Config ---
MODEL_ID_JINA = "jinaai/jina-embeddings-v3"
DIM = 768
TASK = "classification"
BATCH_SIZE = 32

print(f"Encoder: {MODEL_ID_JINA}")
print(f"Task: {TASK} | Dim: {DIM}")
print(f"Device: {DEVICE}")

# --- Load Encoder ---
print("\nLoading Jina v3 encoder...")
encoder = SentenceTransformer(
    MODEL_ID_JINA,
    trust_remote_code=True,
    device=DEVICE,
    truncate_dim=DIM,
)

# --- Encode Function ---
def encode_texts(enc, texts, task, batch_size, dim):
    """Encode texts with Jina v3."""
    return enc.encode(
        texts,
        convert_to_numpy=True,
        show_progress_bar=True,
        batch_size=batch_size,
        normalize_embeddings=True,
        task=task,
        truncate_dim=dim,
    )

# --- Encode All Splits (use RAW data, not preprocessed) ---
print("\nLoading raw dataset for embeddings...")
raw_dataset = load_dataset("tweet_eval", "irony")

print("Encoding train...")
X_train = encode_texts(encoder, raw_dataset["train"]["text"], TASK, BATCH_SIZE, DIM)
y_train = np.array(raw_dataset["train"]["label"], dtype=np.int64)

print("Encoding val...")
X_val = encode_texts(encoder, raw_dataset["validation"]["text"], TASK, BATCH_SIZE, DIM)
y_val = np.array(raw_dataset["validation"]["label"], dtype=np.int64)

print("Encoding test...")
X_test = encode_texts(encoder, raw_dataset["test"]["text"], TASK, BATCH_SIZE, DIM)
y_test = np.array(raw_dataset["test"]["label"], dtype=np.int64)

print(f"Embeddings shape: Train {X_train.shape} | Val {X_val.shape} | Test {X_test.shape}")

# --- Free VRAM ---
del encoder
flush_memory()

# --- Merge train + val ---
X_full_train = np.vstack([X_train, X_val])
y_full_train = np.concatenate([y_train, y_val])
print(f"Full training set: {X_full_train.shape}")

# --- CV Setup ---
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
scorer = make_scorer(f1_score, pos_label=1, average="binary")

# =============================================================================
# XGBOOST
# =============================================================================
print("\n" + "=" * 60)
print("Training XGBoost...")
print("=" * 60)

xgb_params = {
    "n_estimators": [50, 100, 200, 300],
    "learning_rate": [0.005, 0.01, 0.03, 0.05, 0.1],
    "max_depth": [3, 5, 6, 8, 10],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "gamma": [0.0, 0.1, 0.2, 0.5, 1.0],
    "reg_lambda": [0.0, 0.5, 1.0, 2.0],
    "reg_alpha": [0.0, 0.01, 0.05, 0.1, 0.5],
}

xgb_model = xgb.XGBClassifier(
    random_state=SEED,
    eval_metric="logloss",
    tree_method="hist",
    device=DEVICE,
    n_jobs=1,
)

start_xgb = time.time()
xgb_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_params,
    n_iter=100,
    cv=cv,
    scoring=scorer,
    verbose=1,
    random_state=SEED,
    n_jobs=1,
    refit=True,
)
xgb_search.fit(X_full_train, y_full_train)
xgb_train_time = time.time() - start_xgb

best_xgb = xgb_search.best_estimator_
best_xgb_params = xgb_search.best_params_
print(f"XGBoost training complete in {xgb_train_time:.1f}s")

# --- XGBoost Inference ---
start_inf = time.time()
y_pred_xgb = best_xgb.predict(X_test)
xgb_inference_time = time.time() - start_inf

# --- XGBoost Evaluation ---
evaluate_model(y_test, y_pred_xgb, f"XGBoost (Jina v3 {DIM}d)", xgb_inference_time)

# --- Save XGBoost ---
os.makedirs("models", exist_ok=True)
joblib.dump(best_xgb, f"models/xgb_jina_v3_{DIM}.joblib")
print(f"XGBoost saved to models/xgb_jina_v3_{DIM}.joblib")

# =============================================================================
# RANDOM FOREST
# =============================================================================
print("\n" + "=" * 60)
print("Training Random Forest...")
print("=" * 60)

rf_params = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [None, 10, 20, 30, 50],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features": ["sqrt", "log2", None],
    "class_weight": ["balanced"],
}

rf = RandomForestClassifier(random_state=SEED, n_jobs=1)

start_rf = time.time()
rf_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_params,
    n_iter=50,
    cv=cv,
    scoring=scorer,
    verbose=1,
    random_state=SEED,
    n_jobs=-1,
    refit=True,
)
rf_search.fit(X_full_train, y_full_train)
rf_train_time = time.time() - start_rf

best_rf = rf_search.best_estimator_
best_rf_params = rf_search.best_params_
print(f"RF training complete in {rf_train_time:.1f}s")

# --- RF Inference ---
start_inf = time.time()
y_pred_rf = best_rf.predict(X_test)
rf_inference_time = time.time() - start_inf

# --- RF Evaluation ---
evaluate_model(y_test, y_pred_rf, f"Random Forest (Jina v3 {DIM}d)", rf_inference_time)

# --- Save RF ---
joblib.dump(best_rf, f"models/rf_jina_v3_{DIM}.joblib")
print(f"RF saved to models/rf_jina_v3_{DIM}.joblib")

# =============================================================================
# BEST HYPERPARAMETERS SUMMARY
# =============================================================================
print("\n" + "=" * 60)
print("BEST HYPERPARAMETERS")
print("=" * 60)

print("\n--- XGBoost ---")
for param, value in best_xgb_params.items():
    print(f"  {param}: {value}")

print("\n--- Random Forest ---")
for param, value in best_rf_params.items():
    print(f"  {param}: {value}")

print("\n" + "=" * 60)

# --- Cleanup ---
del X_train, X_val, X_test, X_full_train
del best_rf, best_xgb, rf_search, xgb_search
flush_memory()

print("\nExperiment 3 complete!")

EXPERIMENT 3: Jina v3 Embeddings + Classic ML
Encoder: jinaai/jina-embeddings-v3
Task: classification | Dim: 768
Device: cuda

Loading Jina v3 encoder...


flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i


Loading raw dataset for embeddings...
Encoding train...


Batches: 100%|██████████| 90/90 [00:13<00:00,  6.58it/s]


Encoding val...


...

Encoding test...


Batches: 100%|██████████| 25/25 [00:03<00:00,  6.97it/s]


Embeddings shape: Train (2862, 768) | Val (955, 768) | Test (784, 768)
Memory flushed. VRAM ready for the next round.
Full training set: (3817, 768)

Training XGBoost...
Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\anass\Desktop\irony-detection-nlp-benchmark\.venv\Lib\site-packages\xgboost\core.py:774: UserWarning: [22:51:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


XGBoost training complete in 1234.1s

📊 RESULTS: XGBoost (Jina v3 768d)
  Accuracy: 66.84%
  F1-Score (Irony): 61.08%
  Inference Time: 0.04s
----------------------------------------
              precision    recall  f1-score   support

   Not_Irony       0.75      0.68      0.71       473
       Irony       0.57      0.66      0.61       311

    accuracy                           0.67       784
   macro avg       0.66      0.67      0.66       784
weighted avg       0.68      0.67      0.67       784

XGBoost saved to models/xgb_jina_v3_768.joblib

Training Random Forest...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
RF training complete in 818.1s

📊 RESULTS: Random Forest (Jina v3 768d)
  Accuracy: 65.82%
  F1-Score (Irony): 59.15%
  Inference Time: 0.02s
----------------------------------------
              precision    recall  f1-score   support

   Not_Irony       0.73      0.68      0.71       473
       Irony       0.56      0.62      0.59       311

    acc

## Jina Embeddings v3 (256 dimensions)

In [13]:
# =============================================================================
# EXPERIMENT 3b: Classic ML + Jina Embeddings v3 (256 dimensions)
# =============================================================================
# Same approach as Experiment 3, but with reduced dimensionality.
# Tests if Matryoshka truncation to 256d affects performance significantly.
# =============================================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer
import joblib

print("=" * 60)
print("EXPERIMENT 3b: Jina v3 Embeddings + Classic ML (256d)")
print("=" * 60)

# --- Config ---
MODEL_ID_JINA = "jinaai/jina-embeddings-v3"
DIM = 256
TASK = "classification"
BATCH_SIZE = 32

print(f"Encoder: {MODEL_ID_JINA}")
print(f"Task: {TASK} | Dim: {DIM}")
print(f"Device: {DEVICE}")

# --- Load Encoder ---
print("\nLoading Jina v3 encoder...")
encoder = SentenceTransformer(
    MODEL_ID_JINA,
    trust_remote_code=True,
    device=DEVICE,
    truncate_dim=DIM,
)

# --- Encode Function ---
def encode_texts(enc, texts, task, batch_size, dim):
    """Encode texts with Jina v3."""
    return enc.encode(
        texts,
        convert_to_numpy=True,
        show_progress_bar=True,
        batch_size=batch_size,
        normalize_embeddings=True,
        task=task,
        truncate_dim=dim,
    )

# --- Encode All Splits (use RAW data, not preprocessed) ---
print("\nLoading raw dataset for embeddings...")
raw_dataset = load_dataset("tweet_eval", "irony")

print("Encoding train...")
X_train = encode_texts(encoder, raw_dataset["train"]["text"], TASK, BATCH_SIZE, DIM)
y_train = np.array(raw_dataset["train"]["label"], dtype=np.int64)

print("Encoding val...")
X_val = encode_texts(encoder, raw_dataset["validation"]["text"], TASK, BATCH_SIZE, DIM)
y_val = np.array(raw_dataset["validation"]["label"], dtype=np.int64)

print("Encoding test...")
X_test = encode_texts(encoder, raw_dataset["test"]["text"], TASK, BATCH_SIZE, DIM)
y_test = np.array(raw_dataset["test"]["label"], dtype=np.int64)

print(f"Embeddings shape: Train {X_train.shape} | Val {X_val.shape} | Test {X_test.shape}")

# --- Free VRAM ---
del encoder
flush_memory()

# --- Merge train + val ---
X_full_train = np.vstack([X_train, X_val])
y_full_train = np.concatenate([y_train, y_val])
print(f"Full training set: {X_full_train.shape}")

# --- CV Setup ---
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
scorer = make_scorer(f1_score, pos_label=1, average="binary")

# =============================================================================
# XGBOOST
# =============================================================================
print("\n" + "=" * 60)
print("Training XGBoost...")
print("=" * 60)

xgb_params = {
    "n_estimators": [100, 200, 250, 300, 350, 400, 450, 500],
    "learning_rate": [0.005, 0.01, 0.03, 0.05, 0.1],
    "max_depth": [3, 5, 6, 8, 10],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "gamma": [0.0, 0.1, 0.2, 0.5, 1.0],
    "reg_lambda": [0.0, 0.5, 1.0, 2.0],
    "reg_alpha": [0.0, 0.01, 0.05, 0.1, 0.5],
    "max_bin": [128, 256],
}

xgb_model = xgb.XGBClassifier(
    random_state=SEED,
    eval_metric="logloss",
    tree_method="hist",
    device=DEVICE,
    n_jobs=1,
)

start_xgb = time.time()
xgb_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_params,
    n_iter=200,
    cv=cv,
    scoring=scorer,
    verbose=1,
    random_state=SEED,
    n_jobs=1,
    refit=True,
)
xgb_search.fit(X_full_train, y_full_train)
xgb_train_time = time.time() - start_xgb

best_xgb = xgb_search.best_estimator_
best_xgb_params = xgb_search.best_params_
print(f"XGBoost training complete in {xgb_train_time:.1f}s")

# --- XGBoost Inference ---
start_inf = time.time()
y_pred_xgb = best_xgb.predict(X_test)
xgb_inference_time = time.time() - start_inf

# --- XGBoost Evaluation ---
evaluate_model(y_test, y_pred_xgb, f"XGBoost (Jina v3 {DIM}d)", xgb_inference_time)

# --- Save XGBoost ---
os.makedirs("models", exist_ok=True)
joblib.dump(best_xgb, f"models/xgb_jina_v3_{DIM}.joblib")
print(f"XGBoost saved to models/xgb_jina_v3_{DIM}.joblib")

# =============================================================================
# RANDOM FOREST
# =============================================================================
print("\n" + "=" * 60)
print("Training Random Forest...")
print("=" * 60)

rf_params = {
    "n_estimators": [100, 200, 250, 300],
    "max_depth": [None, 10, 30, 50, 70],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features": ["sqrt", "log2", None],
    "class_weight": ["balanced"],
}

rf = RandomForestClassifier(random_state=SEED, n_jobs=1)

start_rf = time.time()
rf_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_params,
    n_iter=70,
    cv=cv,
    scoring=scorer,
    verbose=1,
    random_state=SEED,
    n_jobs=-1,
    refit=True,
)
rf_search.fit(X_full_train, y_full_train)
rf_train_time = time.time() - start_rf

best_rf = rf_search.best_estimator_
best_rf_params = rf_search.best_params_
print(f"RF training complete in {rf_train_time:.1f}s")

# --- RF Inference ---
start_inf = time.time()
y_pred_rf = best_rf.predict(X_test)
rf_inference_time = time.time() - start_inf

# --- RF Evaluation ---
evaluate_model(y_test, y_pred_rf, f"Random Forest (Jina v3 {DIM}d)", rf_inference_time)

# --- Save RF ---
joblib.dump(best_rf, f"models/rf_jina_v3_{DIM}.joblib")
print(f"RF saved to models/rf_jina_v3_{DIM}.joblib")

# =============================================================================
# BEST HYPERPARAMETERS SUMMARY
# =============================================================================
print("\n" + "=" * 60)
print("BEST HYPERPARAMETERS (256 dimensions)")
print("=" * 60)

print("\n--- XGBoost ---")
for param, value in best_xgb_params.items():
    print(f"  {param}: {value}")

print("\n--- Random Forest ---")
for param, value in best_rf_params.items():
    print(f"  {param}: {value}")

print("\n" + "=" * 60)

# --- Cleanup ---
del X_train, X_val, X_test, X_full_train
del best_rf, best_xgb, rf_search, xgb_search
flush_memory()

print("\nExperiment 3b complete!")              

EXPERIMENT 3b: Jina v3 Embeddings + Classic ML (256d)
Encoder: jinaai/jina-embeddings-v3
Task: classification | Dim: 256
Device: cuda

Loading Jina v3 encoder...


flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i


Loading raw dataset for embeddings...
Encoding train...


Batches: 100%|██████████| 90/90 [00:10<00:00,  8.24it/s]


Encoding val...


Batches: 100%|██████████| 30/30 [00:03<00:00,  8.99it/s]


Encoding test...


Batches: 100%|██████████| 25/25 [00:02<00:00,  8.87it/s]


Embeddings shape: Train (2862, 256) | Val (955, 256) | Test (784, 256)
Memory flushed. VRAM ready for the next round.
Full training set: (3817, 256)

Training XGBoost...
Fitting 3 folds for each of 200 candidates, totalling 600 fits
XGBoost training complete in 1694.9s

📊 RESULTS: XGBoost (Jina v3 256d)
  Accuracy: 71.43%
  F1-Score (Irony): 67.44%
  Inference Time: 0.03s
----------------------------------------
              precision    recall  f1-score   support

   Not_Irony       0.81      0.69      0.75       473
       Irony       0.62      0.75      0.67       311

    accuracy                           0.71       784
   macro avg       0.71      0.72      0.71       784
weighted avg       0.73      0.71      0.72       784

XGBoost saved to models/xgb_jina_v3_256.joblib

Training Random Forest...
Fitting 3 folds for each of 70 candidates, totalling 210 fits
RF training complete in 974.5s

📊 RESULTS: Random Forest (Jina v3 256d)
  Accuracy: 65.69%
  F1-Score (Irony): 58.16%
  I

## Analysis: Classic ML vs Fine-tuned Transformers

### Results Summary

| Model | Accuracy | F1 (Irony) | Inference Time | Parameters |
|-------|----------|------------|----------------|------------|
| **Cardiff RoBERTa-2021-124M (SOTA)** | **78.57%** | **75.07%** | 1.72s | 125M |
| SetFit (MPNet) | 74.00% | 72.00% | 0.96s | 109M |
| Cardiff RoBERTa-Base | 73.47% | 62.72% | 9.54s | 125M |
| **XGBoost (Jina v3 256d)** | **71.43%** | **67.44%** | 0.03s | <1M |
| XGBoost (Jina v3 768d) | 66.84% | 61.08% | 0.04s | <1M |
| Random Forest (Jina v3 768d) | 65.82% | 59.15% | 0.02s | <1M |
| Random Forest (Jina v3 256d) | 65.69% | 58.16% | 0.06s | <1M |

---

### Key Finding: XGBoost 256d vs Cardiff Base

| Metric | XGBoost 256d | Cardiff Base | Difference |
|--------|--------------|--------------|------------|
| F1 (Irony) | 67.44% | 62.72% | **+4.7 pts** |
| Accuracy | 71.43% | 73.47% | -2.0 pts |
| Inference | 0.03s | 9.54s | **318x faster** |
| Parameters | <1M | 125M | **125x smaller** |

XGBoost 256d **outperforms** Cardiff RoBERTa-Base on F1 score while being 318x faster and 125x smaller.

---

### Why Classic ML Underperforms vs SOTA

Classic ML models (RF, XGBoost) cannot capture the **contextual and semantic nuances** required for irony detection. Irony relies on subtle linguistic patterns, tone shifts, and implicit meaning that transformers learn through pre-training on massive corpora. Tree-based models only see fixed embeddings without context adaptation.

---

### Why 256d Outperforms 768d (Curse of Dimensionality)

| Dimension | Features | Training Samples | Samples/Feature Ratio |
|-----------|----------|------------------|----------------------|
| 768d | 768 | 3,817 | ~5:1 |
| 256d | 256 | 3,817 | ~15:1 |

With 768 features and only ~3,800 samples, tree-based models suffer from:
- **Overfitting**: Too many features relative to observations
- **Sparse splits**: Trees cannot find robust decision boundaries
- **Noise amplification**: Higher dimensions contain more noise

256 dimensions provide a better signal-to-noise ratio for the limited dataset size.

---

### Jina v3 Matryoshka Embeddings: Relative Performance

Jina v3 uses Matryoshka Representation Learning, allowing truncation to smaller dimensions with minimal quality loss:

| Dimension | Relative Performance | Use Case |
|-----------|---------------------|----------|
| 1024 (Full) | 100% | Maximum precision tasks |
| 768 | ~99.5% | General purpose |
| 512 | ~99.0% | Balanced |
| 256 | ~97.5% | Classification (sufficient) |
| 128 | ~95.0% | Lightweight/mobile |

For **binary classification** tasks like irony detection, 256 dimensions retain sufficient discriminative power while significantly reducing computational overhead.

---

### Conclusion

| Aspect | XGBoost 256d | Verdict |
|--------|--------------|---------|
| **Performance** | 67.44% F1 | Comparable to Cardiff Base, ~10% below SOTA |
| **Speed** | 0.03s inference | 57x faster than SOTA |
| **Size** | <1MB model | 125x smaller than transformers |
| **Complexity** | Simple tree ensemble | No GPU required |
| **Training** | ~28 min | No pre-training needed |

**XGBoost 256d is a strong baseline** for resource-constrained environments where transformer inference is not feasible. It demonstrates that high-quality embeddings + simple classifiers can compete with older fine-tuned models, but cannot match modern SOTA transformers on nuanced NLP tasks like irony detection.

# Large Language Model (`Qwen 2.5-7b-instruct`)

In [ ]:
import json
import requests
import time
import os
from sklearn.metrics import classification_report, accuracy_score
from datasets import load_dataset

# --- CONFIGURATION ---
MODEL = "qwen2.5:7b-instruct"
URL = "http://localhost:11434/api/chat"
OUTPUT_FILE = "resultats_ironie_full.json"
BACKUP_EVERY = 50  # Sauvegarde automatique tous les 50 tweets

# --- 1. CHARGEMENT COMPLET (TRAIN + TEST) ---
print("⏳ Chargement de TOUT le dataset TweetEval (Irony)...")
dataset_train = load_dataset("tweet_eval", "irony", split="train")
dataset_test = load_dataset("tweet_eval", "irony", split="test")

# On fusionne tout dans une seule liste pour la nuit
full_data = []
print("📦 Fusion des données...")
for item in dataset_train:
    full_data.append({"text": item['text'], "label": item['label'], "split": "train"})
for item in dataset_test:
    full_data.append({"text": item['text'], "label": item['label'], "split": "test"})

print(f"✅ Prêt à traiter {len(full_data)} tweets cette nuit.")

# --- 2. FONCTION ROBUSTE ---
def classify_robust(text, max_retries=3):
    payload = {
        "model": MODEL,
        "stream": False,
        "messages": [
            {"role": "system", "content": "You are a strict irony detector. Output JSON only."},
            # Few-Shot Examples (Ta recette secrète)
            {"role": "user", "content": "Text: I love getting stuck in traffic.\nReturn JSON exactly: {\"label\":\"IRONY\"|\"NOT_IRONY\",\"confidence\":0-1}."},
            {"role": "assistant", "content": "{\"label\":\"IRONY\", \"confidence\": 0.99}"},
            {"role": "user", "content": "Text: The weather is beautiful today.\nReturn JSON exactly: {\"label\":\"IRONY\"|\"NOT_IRONY\",\"confidence\":0-1}."},
            {"role": "assistant", "content": "{\"label\":\"NOT_IRONY\", \"confidence\": 0.99}"},
            # Target
            {"role": "user", "content": f"Return JSON exactly: {{\"label\":\"IRONY\"|\"NOT_IRONY\",\"confidence\":0-1}}.\nText: {text}"}
        ],
        "options": {"temperature": 0}
    }
    
    for attempt in range(max_retries):
        try:
            r = requests.post(URL, json=payload, timeout=120)
            r.raise_for_status()
            content = r.json()["message"]["content"]
            
            # Parsing JSON artisanal mais solide
            start = content.find('{')
            end = content.rfind('}') + 1
            if start != -1 and end != -1:
                return json.loads(content[start:end]).get("label", "NOT_IRONY")
            
            # Fallback
            if "IRONY" in content and "NOT" not in content: return "IRONY"
            return "NOT_IRONY"
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"\n⚠️ Échec définitif sur ce tweet : {e}")
                return "ERROR"
            time.sleep(2) # On attend un peu avant de réessayer

# --- 3. EXÉCUTION & SAUVEGARDE CONTINUE ---
results = []
# Si le fichier existe déjà, on reprend (optionnel, ici on écrase pour faire simple)
if os.path.exists(OUTPUT_FILE):
    print(f"⚠️ Le fichier {OUTPUT_FILE} existe déjà, je vais ajouter à la suite...")

print("\n🚀 C'est parti ! Bonne nuit. (Ctrl+C pour arrêter si besoin)\n")

start_time = time.time()
y_true = []
y_pred = []

for i, item in enumerate(full_data):
    text = item['text']
    true_label = "IRONY" if item['label'] == 1 else "NOT_IRONY"
    
    # Classification
    pred_label = classify_robust(text)
    
    # On stocke tout
    result_entry = {
        "id": i,
        "text": text,
        "true_label": true_label,
        "pred_label": pred_label,
        "split": item['split']
    }
    results.append(result_entry)
    
    # Pour les métriques
    if pred_label != "ERROR":
        y_true.append(true_label)
        y_pred.append(pred_label)

    # Affichage minimaliste pour pas bourrer la console
    status = "✅" if pred_label == true_label else "❌"
    print(status, end="", flush=True)
    
    # Sauvegarde intermédiaire (Sécurité)
    if (i + 1) % BACKUP_EVERY == 0:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f" [Sauvegardé {i+1}/{len(full_data)}]", end="", flush=True)

# --- 4. RÉCAPITULATIF AU RÉVEIL ---
# Sauvegarde finale
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

duration = (time.time() - start_time) / 60
print(f"\n\n🏁 FINI ! Durée totale : {duration:.1f} minutes.")
print(f"📁 Résultats complets sauvegardés dans : {OUTPUT_FILE}")

print("\n" + "="*50)
print("📊 RAPPORT FINAL (Train + Test)")
print("="*50)
print(f"Accuracy Globale: {accuracy_score(y_true, y_pred):.2%}")
print(classification_report(y_true, y_pred, labels=["IRONY", "NOT_IRONY"]))

⏳ Chargement de TOUT le dataset TweetEval (Irony)...
📦 Fusion des données...
✅ Prêt à traiter 3646 tweets cette nuit.

🚀 C'est parti ! Bonne nuit. (Ctrl+C pour arrêter si besoin)

✅❌❌❌✅✅✅✅✅✅❌❌✅✅✅❌✅❌✅❌❌❌❌✅✅✅✅✅✅✅✅✅✅✅✅✅✅❌✅❌❌✅✅❌✅✅❌❌❌✅ [Sauvegardé 50/3646]✅✅❌❌❌❌✅✅❌✅❌✅✅❌✅❌❌❌❌❌✅✅✅✅❌✅✅✅✅✅✅❌✅✅✅✅❌❌✅❌❌✅✅✅❌✅✅❌✅✅ [Sauvegardé 100/3646]✅✅✅✅✅✅❌❌❌❌✅✅❌❌✅✅✅✅✅✅❌✅❌❌✅✅❌✅❌✅✅✅✅❌✅❌❌✅✅✅✅✅✅✅❌❌✅✅❌✅ [Sauvegardé 150/3646]❌❌❌✅❌✅✅✅✅✅✅❌✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅❌✅✅✅✅❌✅✅✅✅✅❌✅❌❌✅✅✅❌❌ [Sauvegardé 200/3646]✅❌❌❌✅❌✅✅✅✅✅❌❌✅❌✅✅❌✅❌✅✅❌✅❌❌❌✅❌✅✅❌❌✅❌✅❌✅✅✅✅✅✅✅✅❌❌❌❌❌ [Sauvegardé 250/3646]✅✅✅✅✅✅✅❌✅✅❌❌✅✅❌❌❌✅✅✅✅❌✅❌✅❌✅❌❌✅✅❌❌❌✅❌✅✅✅❌❌✅❌❌❌✅❌✅✅❌ [Sauvegardé 300/3646]❌✅✅❌❌✅❌❌❌✅❌✅✅❌✅❌❌❌✅✅✅❌❌❌✅❌✅✅✅✅✅✅❌❌❌✅✅❌❌✅✅❌❌❌✅❌✅❌✅✅ [Sauvegardé 350/3646]✅✅✅✅✅✅❌✅✅✅✅✅❌❌✅❌❌✅❌✅✅❌❌❌✅✅✅✅✅✅✅✅❌✅✅❌✅✅❌❌✅❌✅✅❌❌✅❌✅❌ [Sauvegardé 400/3646]✅✅❌✅❌✅✅❌❌❌✅✅❌✅✅❌✅✅✅❌❌✅❌❌✅✅❌✅❌❌✅✅✅✅✅✅✅✅❌❌❌✅✅✅✅❌✅✅✅✅ [Sauvegardé 450/3646]✅✅❌✅✅✅❌❌✅❌✅❌✅✅❌✅✅✅✅✅✅✅✅❌✅✅✅✅✅❌✅❌❌❌✅✅❌❌✅❌✅✅✅✅✅✅✅❌❌✅ [Sauvegardé 500/3646]❌✅✅✅❌✅✅❌✅❌❌✅✅✅✅❌❌❌✅✅✅❌✅✅❌❌✅✅✅❌❌❌✅✅❌❌✅❌✅✅✅✅❌✅✅❌❌❌❌✅ [Sauvegardé 550/3646]✅✅❌✅✅✅✅✅✅✅✅✅✅✅❌❌❌✅✅❌✅✅✅❌❌❌❌✅✅

# Dire que c'est le resultat le plus mauvais,  meme si ce modele est un modeles tres populaire comme un LLM inferable en LOCAL,  d'ailleur JINA v4 qui est un  modele d'embedding SOTA, recement sortie (07/2025,  n'oublie pas maintenant  c'est 01/2026 don c c'est un  modele tres recent) est "Built on Qwen/Qwen2.5-VL-3B-Instruct", utilise que, il est meme tres complexe e espace et en  temps. Explique finalement en  2  lignes pourquoi 

E